In [9]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter as gauss
from pyspec import spectrum as spec
from matplotlib.colors import LogNorm
import warnings
import scipy.integrate as integ
warnings.filterwarnings("ignore")
import sys
sys.path.append("/homes/metogra/iufarias/Documents/Scripts/useful")
import romspickle as rp

In [4]:
path='/data/pacific/lrenault/SASI/CROCO/FULL/'
listm=glob(path+'surf*.nc');listm.sort()
# varf=xr.open_dataset(listm[-9:][xmon])
varf=xr.open_mfdataset(listm[1:])

path='/data/pacific/lrenault/SASI/CROCO/SMTH/'
listt=glob(path+'surf*SASI_6h_his_20*_*.nc');listt.sort()
listt.append('/data/pacific/lrenault/SASI/CROCO/SMTH/surf_SASI_6h_his_2012-07.nc')
#vart=xr.open_dataset(listt[xmon])
vart=xr.open_mfdataset(listt[1:])

In [5]:
dxx=111*(np.diff(varf['nav_lat_rho'][:,0]).mean() +np.diff(varf['nav_lon_rho'][0,:]).mean())/2
# dxx=0.00485*111

In [6]:
plt.rcParams['font.size']=15
plt.rcParams['font.sans-serif']='Arial'
plt.rcParams['font.weight']='bold'

In [7]:
import datetime

# t_ind=pd.DatetimeIndex(varf.time_counter.compute())
t_ind=varf.time_counter.groupby("time_counter.dayofyear").mean()
mon_ind=t_ind.dt.month
mes=np.array([12,1,2,3,4,5,6,7])
mes_str=['Dec/11','Jan/12','Feb/12','Mar/12','Apr/12','May/12','Jun/12','Jul/12']

In [11]:
ufnew,vfnew,wfnew=rp.uvw2rho_3d(varf.u,varf.v,varf.w)
utnew,vtnew,wtnew=rp.uvw2rho_3d(vart.u,vart.v,vart.w)

xlat,xlon=varf.nav_lat_rho[1:-1,1:-1],varf.nav_lon_rho[1:-1,1:-1]

In [28]:
ufnew

<xarray.DataArray 'u' (time_counter: 852, y_u: 1518, x_u: 998)>
dask.array<getitem, shape=(852, 1518, 998), dtype=float32, chunksize=(124, 1518, 998), chunktype=numpy.ndarray>
Coordinates:
    time_instant  (time_counter) datetime64[ns] dask.array<chunksize=(124,), meta=np.ndarray>
  * time_counter  (time_counter) datetime64[ns] 2012-01-01T06:00:00 ... 2012-...
Dimensions without coordinates: y_u, x_u

In [40]:
uf=ufnew.groupby("time_counter.dayofyear").mean()[:,30:-30,30:850]
vf=vfnew.groupby("time_counter.dayofyear").mean()[:,30:-30,30:850]
ut=utnew.groupby("time_counter.dayofyear").mean()[:,30:-30,30:850]
vt=vtnew.groupby("time_counter.dayofyear").mean()[:,30:-30,30:850]

In [41]:
lati=np.linspace(xlat.min(),xlat.max(),len(xlat[:,0]))
loni=np.linspace(xlon.min(),xlon.max(),len(xlon[0,:]))

mloni,mlati=np.meshgrid(loni,lati)

In [42]:
uf.shape

(214, 1458, 820)

In [43]:
ufi=griddata((np.ravel(xlon[30:-30,30:850]),np.ravel(xlat[30:-30,30:850])),np.ravel(uf),(mloni,mlati),method='linear');
vfi=griddata((np.ravel(xlon[30:-30,30:850]),np.ravel(xlat[30:-30,30:850])),np.ravel(vf),(mloni,mlati),method='linear');

uti=griddata((np.ravel(xlon[30:-30,30:850]),np.ravel(xlat[30:-30,30:850])),np.ravel(ut),(mloni,mlati),method='linear');
vti=griddata((np.ravel(xlon[30:-30,30:850]),np.ravel(xlat[30:-30,30:850])),np.ravel(vt),(mloni,mlati),method='linear');

ValueError: different number of values and points

In [ ]:
xd=np.arange(400,500)
yd=np.arange(200,1300)
plt.contourf(mloni[yd,:][:,xd],mlati[yd,:][:,xd],ufi[yd,:][:,xd])